In [1]:
import spacy
import pandas as pd
from spacy.matcher import Matcher

In [2]:
# todo: set as env variable for raw main table
rootpath = "/hdd/work/d4ad_standardization/"
filepath = "./D4AD_Standardization/data/raw/etpl_all_programsJune3.xls"

columns = [
    "NAME",
    "NAME_1",
    "DESCRIPTION",
    "PREREQUISITES",
    "FEATURESDESCRIPTION",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE"
]

df = pd.read_excel(rootpath + filepath, usecols=columns)

nlp = spacy.load("en_core_web_lg")
print('done')

done


In [3]:
# for test dev purposes, let's focus on a really small subset
BATCH_SIZE = 50
N = 20
random_state = 42
small_df = df.sample(n=N, random_state=random_state)
print('done')

done


In [17]:
def batches_of_docs(df, column_index=0, nlp=nlp, batch_size=BATCH_SIZE, disable=["parser","ner", "entity_linker"]):
    yield nlp.pipe(df.iloc[:,column_index].values,
                   batch_size=batch_size,
                   disable=disable)

def contents_of(matches, doc, matcher_spans_content=False):
    # if matcher_spans_content=False then the matcher
    # indicates where content is not and we span the
    # regions excluded by the matcher
    print(doc)
    if not matcher_spans_content:
        match_start = 0
        for match in matches:
            match_end = match[1]
            print("\t", doc[match_start:match_end])
            yield doc[match_start:match_end]
            match_start = match[2]
        print('\t', doc[match_start:])
        yield doc[match_start:]


patterns =\
    [
        # these break up small_df.iloc[0] into unstandardized tokens
        #[{'POS': 'PUNCT'}],  # fails in later samples
        [{'POS': 'CCONJ'}],
        # modifiction that breaks up small_df.iloc[7]
        [{'ORTH': '/'}],
        # modifiction that combines small_df.iloc[15], [1]
        [{'ORTH': ','}],
        # modifiction seen generally past 50 or os
        [{'ORTH': ';'}],
        # TODO: fix this to work, i could be special casing too early/improperly
        # modifiction seen random_state*2
        [{'IS_SPACE': True}], # captures present spaces after tokenizations
    ]

patterns =\
    [
        [{'ORTH': '-'}],
    ]


matcher = Matcher(nlp.vocab)
matcher.add("DoNotStandardize", patterns)


for a_batch in batches_of_docs(small_df):
    print('on a batch ...')
    for doc, matches in matcher.pipe(a_batch, return_matches=True, batch_size=50):
        for content in contents_of(matches, doc):
            pass
    """
    for match in matcher.pipe(batch, batch_size=50):
        print(' had a match ... ')
        for content in contents_of(match):
            print(content)
    """

on a batch ...
Star Technical Institute - Edison - PVS
	 Star Technical Institute
	 Edison
	 PVS
New Horizons of Central and Northern NJ, Inc. - Iselin - PVS
	 New Horizons of Central and Northern NJ, Inc.
	 Iselin
	 PVS
KeySkills Learning, Inc. - Maywood
	 KeySkills Learning, Inc.
	 Maywood
Beyond Today - TP
	 Beyond Today
	 TP
Rutgers Center for Innovation Education
	 Rutgers Center for Innovation Education
North Jersey Massage Training Center
	 North Jersey Massage Training Center
Essex County College Credit
	 Essex County College Credit
Rowan College at Gloucester County Division of Continuing Ed - WIA Title 2
	 Rowan College at Gloucester County Division of Continuing Ed
	 WIA Title 2
Raritan Valley Community College - Non Credit
	 Raritan Valley Community College
	 Non Credit
Newark Business Training Institute - TP
	 Newark Business Training Institute
	 TP
ACECS, Inc. - PCS
	 ACECS, Inc.
	 PCS
Rowan College at Burlington County - Non-credit
	 Rowan College at Burlington County
	 